In [1]:
import cv2 
import numpy as np
from keras.models import load_model
from PIL import Image

Using TensorFlow backend.


In [2]:
# Overlays text of classification from model.
def overlayText(txt,x,y):
    cv2.putText(img, txt, (x,y), cv2.FONT_HERSHEY_TRIPLEX,1,(0,255,0))
    
# Format face in bounds.
def apply_offsets(face_coordinates, offsets):
    x, y, width, height = face_coordinates
    x_off, y_off = offsets
    return (x - x_off, x + width + x_off, y - y_off, y + height + y_off)

In [3]:
# Built-in Camera
cap = cv2.VideoCapture(0)

# Facial detector from OpenCV.
face_cascade = cv2.CascadeClassifier('/Users/mattmartin/Desktop/DS 4440/Project/motion/data/haarcascade_frontalface_default.xml')


# Our model.
emotion_model_path = '/Users/mattmartin/Desktop/DS 4440/Project/motion/data/vgg16_64.h5'


# Load Model
emotion_classifier = load_model(emotion_model_path, compile=False)
emotion_target_size = emotion_classifier.input_shape[1:3]

emotion_offsets = (20, 40)
ID = -1
frame_counter = 0
face_counter = 0

# # Map emotion IDs to Names
emotion_labels = {0: 'angry', 1: 'disgust', 2: 'fear', 3: 'happy',
                4: 'sad', 5: 'surprise', 6: 'neutral'}

emotions_seen = []

run = True
while (run):
    cur_faces = 0
    frame_counter += 1
    if (frame_counter % 1) == 0:
        ret, img = cap.read()
        if not ret:
            break

        gray_image = cv2.cvtColor(img, cv2.COLOR_BGR2GRAY)
        faces = face_cascade.detectMultiScale(gray_image,1.3,5)

        for (x,y,w,h) in faces:
            img = cv2.rectangle(img,(x, y),(x+w, y+h),(48,0,0),2)
            x1, x2, y1, y2 = apply_offsets((x,y,w,h), emotion_offsets)
            cur_faces = len(faces)
            
            gray_face = gray_image[y1:y2, x1:x2]
            try:
                gray_face = cv2.resize(gray_face, (emotion_target_size))
            except:
                continue
            
            gray_face = np.expand_dims(gray_face, 0)

            gray_face = np.expand_dims(gray_face, -1)

            emotion_prediction = emotion_classifier.predict(gray_face)
            emotion_label_arg = np.argmax(emotion_prediction)
            emotion_text = emotion_labels[emotion_label_arg]
            emotions_seen.append(emotion_text)
            overlayText(emotion_text, x, y)

            
        # Displays the number of faces currently in the screen
        overlayText("Current Faces: " + str(cur_faces), 920, 80)
        # Display the current frame
        cv2.imshow('Live Recording', img)

        if cv2.waitKey(1) & 0xFF == ord('q'):
            run = False
            break
            
                    
# Release the capture
cap.release()
cv2.destroyAllWindows()



Instructions for updating:
Colocations handled automatically by placer.
hi
hi me


In [41]:
cap.release()
cv2.destroyAllWindows()